# Projeto de Bases de Dados - Parte 3

### Docente Responsável

Prof. Flávio Martins

### Grupo 21
<dl>
    <dt>HH horas (33%)</dt>
    <dd>ist1100293 Carlos Felgueiras</dd>
    <dt>HH horas (33%)</dt>
    <dd>ist1102556 Daniel Carvalho</dd>
    <dt>HH horas (34%)</dt>
    <dd>ist1102597 Duarte Sousa</dd>
<dl>

In [ ]:
%load_ext sql
%sql postgresql://db:db@postgres/db
%config SqlMagic.displaylimit = None

# Empresa de comércio online

## 0. Carregamento da Base de Dados

Carregue o esquema de Base de Dados apresentado no Anexo A.

In [ ]:
%%sql

DROP TABLE IF EXISTS customer CASCADE;
DROP TABLE IF EXISTS orders CASCADE;
DROP TABLE IF EXISTS pay CASCADE;
DROP TABLE IF EXISTS employee CASCADE;
DROP TABLE IF EXISTS process CASCADE;
DROP TABLE IF EXISTS department CASCADE;
DROP TABLE IF EXISTS workplace CASCADE;
DROP TABLE IF EXISTS works CASCADE;
DROP TABLE IF EXISTS office CASCADE;
DROP TABLE IF EXISTS warehouse CASCADE;
DROP TABLE IF EXISTS product CASCADE;
DROP TABLE IF EXISTS contains CASCADE;
DROP TABLE IF EXISTS supplier CASCADE;
DROP TABLE IF EXISTS delivery CASCADE;

CREATE TABLE customer(
cust_no INTEGER PRIMARY KEY,
name VARCHAR(80) NOT NULL,
email VARCHAR(254) UNIQUE NOT NULL,
phone VARCHAR(15),
address VARCHAR(255)
);

CREATE TABLE orders(
order_no INTEGER PRIMARY KEY,
cust_no INTEGER NOT NULL REFERENCES customer,
date DATE NOT NULL
--order_no must exist in contains
);

CREATE TABLE pay(
order_no INTEGER PRIMARY KEY REFERENCES orders,
cust_no INTEGER NOT NULL REFERENCES customer
);

CREATE TABLE employee(
ssn VARCHAR(20) PRIMARY KEY,
TIN VARCHAR(20) UNIQUE NOT NULL,
bdate DATE,
name VARCHAR NOT NULL
--age must be >=18
);

CREATE TABLE process(
ssn VARCHAR(20) REFERENCES employee,
order_no INTEGER REFERENCES orders,
PRIMARY KEY (ssn, order_no)
);

CREATE TABLE department(
name VARCHAR PRIMARY KEY
);

CREATE TABLE workplace(
address VARCHAR PRIMARY KEY,
lat NUMERIC(8, 6) NOT NULL,
long NUMERIC(9, 6) NOT NULL,
UNIQUE(lat, long)
--address must be in warehouse or office but not both
);

CREATE TABLE office(
address VARCHAR(255) PRIMARY KEY REFERENCES workplace
);

CREATE TABLE warehouse(
address VARCHAR(255) PRIMARY KEY REFERENCES workplace
);

CREATE TABLE works(
ssn VARCHAR(20) REFERENCES employee,
name VARCHAR(200) REFERENCES department,
address VARCHAR(255) REFERENCES workplace,
PRIMARY KEY (ssn, name, address)
);

CREATE TABLE product(
SKU VARCHAR(25) PRIMARY KEY,
name VARCHAR(200) NOT NULL,
description VARCHAR,
price NUMERIC(10, 2) NOT NULL,
ean NUMERIC(13) UNIQUE
);

CREATE TABLE contains(
order_no INTEGER REFERENCES orders,
SKU VARCHAR(25) REFERENCES product,
qty INTEGER,
PRIMARY KEY (order_no, SKU)
);

CREATE TABLE supplier(
TIN VARCHAR(20) PRIMARY KEY,
name VARCHAR(200),
address VARCHAR(255),
SKU VARCHAR(25) REFERENCES product,
date DATE
);

CREATE TABLE delivery(
address VARCHAR(255) REFERENCES warehouse,
TIN VARCHAR(20) REFERENCES supplier,
PRIMARY KEY (address, TIN)
);

Crie as instruções para o seu preenchimento de forma consistente, garantindo que todas as consultas SQL e OLAP, apresentadas mais adiante, produzam um resultado não vazio. 

In [ ]:
%%sql
-- populate.sql

INSERT INTO customer VALUES (1, 'Pedro', 'pedrofrigolet@gmail.com', '+351 912345678', 'Rua da Boa Nova nº39, 4050-101 Ermesinde');
INSERT INTO customer VALUES (2, 'Miguel', 'migueltavares@gmail.com', '+351 942069697', 'Travessa da Esperança nº14 2ºD, 4410-490 Rebordosa');
INSERT INTO customer VALUES (3, 'José', 'zedopipo@gmail.com', '+351 918008576', 'Rua dos Lusíadas nº14, 2955-108 Pinhal Novo');

INSERT INTO employee VALUES (10293847562, 201928376, '2000-06-15', 'João');
INSERT INTO employee VALUES (91827364503, 282736579, '2001-03-15', 'Carlos');
INSERT INTO employee VALUES (10243347562, 200923356, '2002-04-15', 'Marcos');

INSERT INTO department VALUES('Recursos Humanos');
INSERT INTO department VALUES('Logística');
INSERT INTO department VALUES('Comercial');

INSERT INTO workplace VALUES('Rua Cid Tomar 49, 2830-573 Palhais', 38.623246, -9.047482);
INSERT INTO workplace VALUES('Avenida Lago 23, 2765-099 Cascais', 38.705305, -9.377783);
INSERT INTO workplace VALUES('Rua Pé Mouro Armz 108, 2735-542 Sintra', 38.770582, -9.287598);
INSERT INTO workplace VALUES('Av. Ten-Cel. Melo Antunes, 4820-398 Fafe', 41.454987, -8.178397);

INSERT INTO office VALUES('Rua Cid Tomar 49, 2830-573 Palhais');
INSERT INTO office VALUES('Av. Ten-Cel. Melo Antunes, 4820-398 Fafe');

INSERT INTO warehouse VALUES('Avenida Lago 23, 2765-099 Cascais');
INSERT INTO warehouse VALUES('Rua Pé Mouro Armz 108, 2735-542 Sintra');

INSERT INTO orders VALUES(1, 2, '2022-11-30');
INSERT INTO orders VALUES(2, 3, '2022-12-07');
INSERT INTO orders VALUES(3, 1, '2022-12-24');
INSERT INTO orders VALUES(4, 3, '2023-01-05');
INSERT INTO orders VALUES(5, 2, '2023-01-17');
INSERT INTO orders VALUES(6, 1, '2023-01-21');
INSERT INTO orders VALUES(7, 1, '2023-02-12');
INSERT INTO orders VALUES(8, 2, '2023-03-19');
INSERT INTO orders VALUES(9, 3, '2023-04-17');

INSERT INTO product VALUES('TS123456', 'Água Molhadinha, 50cl', 'Água mineral natural da Serra da Estrela. Com pH de 7.0, perfeita para a saude. Produto Nacional', 0.50);
INSERT INTO product VALUES('KS944RUR', 'Carregador Apple iPhone', 'Carregador lightning de 40W com fio de 40cm.', 90, 789870254652);
INSERT INTO product VALUES('KS93528TUT', 'Bolachas Oreo', 'Bolachas de chocolate com recheio de creme.', 1.50);
INSERT INTO product VALUES('KA17120203', 'Red Bull, 25cl', 'Bebida energética com cafeína, baixa em açúcar.', 2.50);
INSERT INTO product VALUES('HM21KMPW', 'Portátil Gaming ASUS Rog Strix', 'Intel Core i7, 32Gb Ram, RTX 4080, SSD NVMe 1Tb', 2389.99);
INSERT INTO product VALUES('X00299X9JF', 'Fortnite 1500 VBUCKS Gift Card', 'Código de acesso a 1500 VBUCKS. Compatível com Windows, Mac, iOS, Android, PlayStation, XBOX, Nintendo Switch.', 0.50);

INSERT INTO supplier VALUES(122365939, 'Av. Santo Condestável, 1950-011 Lisboa', 'Molhadinha, Lda.', 'TS123456', '2024-01-01');
INSERT INTO supplier VALUES(135238432, 'Campo Pequeno, 1000-082 Lisboa', 'Apple', 'KS944RUR', '2024-02-01');
INSERT INTO supplier VALUES(159438584, 'Praça Duque de Saldanha 1, 1050-094 Lisboa', 'Oreo', 'KS93528TUT', '2023-12-31');
INSERT INTO supplier VALUES(154294955, 'Av. da Liberdade 240 2º Esq, 1250-148 Lisboa', 'Red Bull', 'KA17120203', '2024-01-01');
INSERT INTO supplier VALUES(165439089, 'Praça Martim Moniz 39, 1150-052 Lisboa', 'ASUS', 'HM21KMPW', '2025-01-01');
INSERT INTO supplier VALUES(194355853, 'Rua do Grilo 84, 1950-144 Lisboa', 'Epic Games', 'X00299X9JF', '2024-12-31');

INSERT INTO works VALUES(10293847562, 'Logística', 'Rua Cid Tomar 49, 2830-573 Palhais');
INSERT INTO works VALUES(10293847562, 'Recursos Humanos', 'Avenida Lago 23, 2765-099 Cascais');
INSERT INTO works VALUES(91827364503, 'Comercial', 'Rua Pé Mouro Armz 108, 2735-542 Sintra');
INSERT INTO works VALUES(10243347562, 'Logística', 'Av. Ten-Cel. Melo Antunes, 4820-398 Fafe');
INSERT INTO works VALUES(10243347562, 'Comercial', 'Rua Cid Tomar 49, 2830-573 Palhais');

INSERT INTO process VALUES(91827364503, 1);
INSERT INTO process VALUES(91827364503, 2);
INSERT INTO process VALUES(91827364503, 3);
INSERT INTO process VALUES(91827364503, 4);
INSERT INTO process VALUES(91827364503, 5);
INSERT INTO process VALUES(91827364503, 6);
INSERT INTO process VALUES(10243347562, 7);
INSERT INTO process VALUES(10243347562, 8);
INSERT INTO process VALUES(10243347562, 9);

INSERT INTO pay VALUES(2, 3);
INSERT INTO pay VALUES(4, 3);
INSERT INTO pay VALUES(5, 2);
INSERT INTO pay VALUES(8, 2);

INSERT INTO contains VALUES(1, 'TS123456', 16);
INSERT INTO contains VALUES(1, 'KA17120203', 3);
INSERT INTO contains VALUES(2, 'HM21KMPW', 1);
INSERT INTO contains VALUES(2, 'X00299X9JF', 3);
INSERT INTO contains VALUES(3, 'KS93528TUT', 1);
INSERT INTO contains VALUES(3, 'KA17120203', 1);
INSERT INTO contains VALUES(4, 'KA17120203', 40);
INSERT INTO contains VALUES(5, 'HM21KMPW', 2);
INSERT INTO contains VALUES(5, 'KS944RUR', 2);
INSERT INTO contains VALUES(6, 'KS944RUR', 1);
INSERT INTO contains VALUES(7, 'TS123456', 2);
INSERT INTO contains VALUES(7, 'KA17120203', 1);
INSERT INTO contains VALUES(7, 'KS93528TUT', 1);
INSERT INTO contains VALUES(8, 'HM21KMPW', 2);
INSERT INTO contains VALUES(9, 'X00299X9JF', 5);

INSERT INTO delivery VALUES('Avenida Lago 23, 2765-099 Cascais', '122365939');
INSERT INTO delivery VALUES('Avenida Lago 23, 2765-099 Cascais', '135238432');
INSERT INTO delivery VALUES('Avenida Lago 23, 2765-099 Cascais', '159438584');
INSERT INTO delivery VALUES('Rua Pé Mouro Armz 108, 2735-542 Sintra', '154294955');
INSERT INTO delivery VALUES('Rua Pé Mouro Armz 108, 2735-542 Sintra', '165439089');
INSERT INTO delivery VALUES('Rua Pé Mouro Armz 108, 2735-542 Sintra', '194355853');

## 1. Restrições de Integridade

Apresente o código para implementar as seguintes restrições de integridade, se necessário, com recurso a extensões procedimentais SQL (Stored Procedures e Triggers):

(RI-1) Nenhum empregado pode ter menos de 18 anos de idade

In [ ]:
%%sql
-- (RI-1)

CREATE OR REPLACE FUNCTION underage_employee_func() RETURNS TRIGGER AS
$$
BEGIN
    IF AGE(NEW.bdate) < interval '18 years' THEN
        RAISE EXCEPTION 'O empregado com SSN % não é maior de idade.',
            NEW.ssn;
    END IF;
    RETURN NEW;
END
$$ LANGUAGE plpgsql;

DROP TRIGGER IF EXISTS underage_employee ON employee;

CREATE CONSTRAINT TRIGGER underage_employee AFTER INSERT OR UPDATE ON employee
    FOR EACH ROW EXECUTE FUNCTION underage_employee_func();
    


In [ ]:
%%sql

ALTER TABLE employee ADD CHECK (AGE(bdate) >= interval '18 years');

In [ ]:
%%sql

INSERT INTO employee VALUES (1029384356, 201528376, '2020-06-15', 'João');

(RI-2) Um 'Workplace' é obrigatoriamente um 'Office' ou 'Warehouse' mas não pode ser ambos

In [ ]:
%%sql
-- (RI-2)

CREATE OR REPLACE FUNCTION workplace_type_func() RETURNS TRIGGER AS
$$
DECLARE in_warehouses VARCHAR;
DECLARE in_offices VARCHAR;
BEGIN
    SELECT address INTO in_warehouses FROM warehouse WHERE address=NEW.address;
    SELECT address INTO in_offices FROM office WHERE address=NEW.address;
    
    IF (in_warehouses IS NULL) AND (in_offices IS NULL) THEN
        RAISE EXCEPTION 'O workplace %s tem de ser office ou warehouse', NEW.address;
    END IF;
    
    IF (in_warehouses IS NOT NULL) AND (in_offices IS NOT NULL) THEN
        RAISE EXCEPTION 'O workplace %s não pode ser office e warehouse ao mesmo tempo', NEW.address;
    END IF;
        
    RETURN NEW;
END
$$ LANGUAGE plpgsql;

DROP TRIGGER IF EXISTS workplace_type ON workplace;

CREATE CONSTRAINT TRIGGER workplace_type AFTER INSERT OR UPDATE ON workplace DEFERRABLE INITIALLY DEFERRED
    FOR EACH ROW EXECUTE FUNCTION workplace_type_func();

(RI-3) Uma 'Order' tem de figurar obrigatoriamente em 'Contains'.

In [ ]:
%%sql
-- (RI-3)

CREATE OR REPLACE FUNCTION order_contains_func() RETURNS TRIGGER AS
$$
DECLARE in_contains INTEGER;
BEGIN
    SELECT DISTINCT order_no INTO in_contains FROM contains WHERE order_no=NEW.order_no;
    
    IF (in_contains IS NULL) THEN
        RAISE EXCEPTION 'A order % precisa de conter pelo menos um produto', NEW.order_no;
    END IF;
        
    RETURN NEW;
END
$$ LANGUAGE plpgsql;

DROP TRIGGER IF EXISTS order_contains ON orders;

CREATE CONSTRAINT TRIGGER order_contains AFTER INSERT OR UPDATE ON orders DEFERRABLE INITIALLY DEFERRED
    FOR EACH ROW EXECUTE FUNCTION order_contains_func();

In [ ]:
%%sql

INSERT INTO orders VALUES (13, 1, '2023-05-20');

## 2. Consultas SQL

Apresente a consulta SQL mais sucinta para cada uma das seguintes questões

1) Qual o número e nome do(s) cliente(s) com maior valor total de encomendas pagas?  

In [ ]:
%%sql

WITH q AS (
    SELECT
        cust_no,
        c.name cust_name,
        qty,
        price
    FROM
        customer c
    NATURAL JOIN
        pay
    NATURAL JOIN
        orders
    NATURAL JOIN
        contains
    INNER JOIN
        product USING (sku)
)
SELECT
    cust_no,
    cust_name
FROM
    q
GROUP BY
    cust_no, cust_name
HAVING
    sum(qty*price) >= ALL(
        SELECT
            sum(qty*price)
        FROM
            q
        GROUP BY
            cust_no
    );
    

2. Qual o nome dos empregados que processaram encomendas em todos os dias de 2022 em que houve encomendas?

In [ ]:
%%sql

SELECT
    e.name
FROM
    employee e
WHERE
    NOT EXISTS (
        SELECT
            date
        FROM
            orders
        WHERE
            EXTRACT(YEAR FROM date) = '2022'
        EXCEPT
        SELECT
            date
        FROM
            employee
        NATURAL JOIN
            process
        NATURAL JOIN
            orders
        WHERE
            ssn = e.ssn
    );

3. Quantas encomendas foram realizadas mas não pagas em cada mês de 2022?

In [ ]:
%%sql

SELECT
   to_char(to_date(CONCAT('01-', generate_series, '-2000'), 'DD-MM-YYYY'), 'Month') as month,
   count(date)
FROM
    generate_series(1, 12)
LEFT JOIN
    (SELECT
         order_no,
         date
     FROM
         orders
     WHERE
         order_no NOT IN (
            SELECT
                 order_no
            FROM
                 pay)
             AND
             EXTRACT(YEAR FROM date) = '2022') o
    ON (generate_series = EXTRACT(MONTH FROM o.date))
GROUP BY generate_series;

## 3. Vistas

Crie uma vista que resuma as informações mais importantes sobre as vendas de produtos, combinando informações de diferentes tabelas do esquema de base de dados. A vista deve ter o seguinte esquema:

product_sales(sku, order_no, qty, total_price, year, month, day_of_month, day_of_week, city)

In [ ]:
%%sql

DROP VIEW IF EXISTS product_sales;

CREATE OR REPLACE VIEW product_sales
AS
    SELECT
        c.sku,
        c.order_no,
        qty,
        qty*price total_price,
        EXTRACT(YEAR FROM date) years,
        EXTRACT(MONTH FROM date) months,
        EXTRACT(DAY FROM date) day_of_month,
        to_char(date, 'Day') day_of_week,
        SUBSTRING(address, '\d ([a-zA-Z ]*)$') city
    FROM
        customer
    NATURAL JOIN
        pay
    NATURAL JOIN
        orders
    NATURAL JOIN
        contains c
    INNER JOIN
        product USING (sku);

In [ ]:
%%sql

SELECT * FROM product_sales;

## 4. Desenvolvimento de Aplicação

### Explicação da arquitetura da aplicação web, incluindo um link para uma versão de trabalho e as relações entre os vários ficheiros na pasta web/arquivos

...

## 5. Consultas OLAP

Usando a vista desenvolvida para a Questão 3, escreva duas consultas SQL que permitam analisar:

1. As quantidade e valores totais de venda de cada produto em 2022, globalmente, por cidade, por mês, dia do mês e dia da semana

In [ ]:
%%sql
-- SELECT ...

2. O valor médio diário das vendas de todos os produtos em 2022, globalmente, por mês e dia da semana

In [ ]:
%%sql
-- SELECT ...

## 6. Índices

Indique, com a devida justificação, que tipo de índice(s), sobre qual(is) atributo(s) e sobre qual(is) tabela(s) faria sentido criar, de forma a agilizar a execução de cada uma das seguintes consultas: 

### 6.1
SELECT order_no
FROM orders 
JOIN contains USING (order_no) 
JOIN product USING (SKU) 
WHERE price > 50 AND 
EXTRACT(YEAR FROM date) = 2023

In [ ]:
%%sql

DROP INDEX IF EXISTS orders_year_idx;

CREATE INDEX orders_year_idx ON orders USING hash (EXTRACT(YEAR FROM date));

DROP INDEX IF EXISTS product_price_idx;

CREATE INDEX product_price_idx ON product (price);

### Tipo de Índice, Atributos & Justificação

...

### 6.2
SELECT order_no, SUM(qty*price)
FROM contains 
JOIN product USING (SKU) 
WHERE name LIKE ‘A%’ 
GROUP BY order_no;

### Tipo de Índice, Atributos & Justificação

...